In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 航班数据
feature = pd.read_table('/Users/tuyu/000000毕业设计/Flight_Delay/Train_Feature/final_feature.csv',sep=',',encoding='gb2312')
feature.head()

,航班编号,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,...,本时段实际进港延误航班数,本时段离港延误时长,本时段离港平均延误时长,本时段离港延误率,上一时段离港平均延误时长,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,标签
0,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,CZ3295,16801,16801,0,1,1.166667,NaN,NaN,0.324175,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [2]:
feature0 = feature[feature['标签'] == 0]
feature0new = feature0.sample(frac=0.15, replace=False, random_state=None,axis=0)
feature1 = feature[feature['标签'] == 1]
featureGBDT = pd.concat([feature0new,feature1],axis=0, ignore_index=True)

In [3]:
feature0SVM = feature0new.sample(frac=0.15, replace=False, random_state=None,axis=0)
feature1SVM = feature1.sample(frac=0.15, replace=False, random_state=None,axis=0)
featureSVM = pd.concat([feature0SVM,feature1SVM],axis=0, ignore_index=True)

In [4]:
del(featureGBDT['本时段离港平均延误时长'])
del(featureGBDT['上一时段离港平均延误时长'])

In [5]:
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
featureGBDT['起飞间隔'] = featureGBDT['起飞间隔'].fillna(0)
featureGBDT['前序延误'] = featureGBDT['前序延误'].fillna(0)

In [6]:
featureGBDT['出发机场天气得分'] = featureGBDT['出发机场天气得分'].interpolate()
featureGBDT['到达机场天气得分'] = featureGBDT['到达机场天气得分'].interpolate()
featureGBDT['平均延误时间'] = featureGBDT['平均延误时间'].interpolate()
featureGBDT['最大延误时间'] = featureGBDT['最大延误时间'].interpolate()
featureGBDT['延误时间中位数'] = featureGBDT['延误时间中位数'].interpolate()
featureGBDT['延误时间标准差'] = featureGBDT['延误时间标准差'].interpolate()


In [7]:

featureGBDT['本时段计划离港航班数'] = featureGBDT['本时段计划离港航班数'].fillna(featureGBDT['本时段计划离港航班数'].mean())
featureGBDT['本时段实际离港航班数'] = featureGBDT['本时段实际离港航班数'].fillna(featureGBDT['本时段实际离港航班数'].mean())
featureGBDT['本时段进港延误时长'] = featureGBDT['本时段进港延误时长'].fillna(featureGBDT['本时段进港延误时长'].mean())
featureGBDT['本时段实际进港延误航班数'] = featureGBDT['本时段实际进港延误航班数'].fillna(featureGBDT['本时段实际进港延误航班数'].mean())
featureGBDT['本时段离港延误时长'] = featureGBDT['本时段离港延误时长'].fillna(featureGBDT['本时段离港延误时长'].mean())
featureGBDT['本时段离港延误率'] = featureGBDT['本时段离港延误率'].fillna(featureGBDT['本时段离港延误率'].mean())
featureGBDT['上一时段进港延误时长'] = featureGBDT['上一时段进港延误时长'].fillna(featureGBDT['上一时段进港延误时长'].mean())
featureGBDT['上一时段离港延误时长'] = featureGBDT['上一时段离港延误时长'].fillna(featureGBDT['上一时段离港延误时长'].mean())
featureGBDT['上一时段实际进港航班数'] = featureGBDT['上一时段实际进港航班数'].fillna(featureGBDT['上一时段实际进港航班数'].mean())
featureGBDT['上一时段进港延误率'] = featureGBDT['上一时段进港延误率'].fillna(featureGBDT['上一时段进港延误率'].mean())



In [8]:
del(featureSVM['本时段离港平均延误时长'])
del(featureSVM['上一时段离港平均延误时长'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
featureSVM['起飞间隔'] = featureSVM['起飞间隔'].fillna(0)
featureSVM['前序延误'] = featureSVM['前序延误'].fillna(0)
featureSVM['出发机场天气得分'] = featureSVM['出发机场天气得分'].interpolate()
featureSVM['到达机场天气得分'] = featureSVM['到达机场天气得分'].interpolate()
featureSVM['平均延误时间'] = featureSVM['平均延误时间'].interpolate()
featureSVM['最大延误时间'] = featureSVM['最大延误时间'].interpolate()
featureSVM['延误时间中位数'] = featureSVM['延误时间中位数'].interpolate()
featureSVM['延误时间标准差'] = featureSVM['延误时间标准差'].interpolate()
featureSVM['本时段计划离港航班数'] = featureSVM['本时段计划离港航班数'].fillna(featureSVM['本时段计划离港航班数'].mean())
featureSVM['本时段实际离港航班数'] = featureSVM['本时段实际离港航班数'].fillna(featureSVM['本时段实际离港航班数'].mean())
featureSVM['本时段进港延误时长'] = featureSVM['本时段进港延误时长'].fillna(featureSVM['本时段进港延误时长'].mean())
featureSVM['本时段实际进港延误航班数'] = featureSVM['本时段实际进港延误航班数'].fillna(featureSVM['本时段实际进港延误航班数'].mean())
featureSVM['本时段离港延误时长'] = featureSVM['本时段离港延误时长'].fillna(featureSVM['本时段离港延误时长'].mean())
featureSVM['本时段离港延误率'] = featureSVM['本时段离港延误率'].fillna(featureSVM['本时段离港延误率'].mean())
featureSVM['上一时段进港延误时长'] = featureSVM['上一时段进港延误时长'].fillna(featureSVM['上一时段进港延误时长'].mean())
featureSVM['上一时段离港延误时长'] = featureSVM['上一时段离港延误时长'].fillna(featureSVM['上一时段离港延误时长'].mean())
featureSVM['上一时段实际进港航班数'] = featureSVM['上一时段实际进港航班数'].fillna(featureSVM['上一时段实际进港航班数'].mean())
featureSVM['上一时段进港延误率'] = featureSVM['上一时段进港延误率'].fillna(featureSVM['上一时段进港延误率'].mean())



In [9]:
del(feature['本时段离港平均延误时长'])
del(feature['上一时段离港平均延误时长'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
feature['起飞间隔'] = feature['起飞间隔'].fillna(0)
feature['前序延误'] = feature['前序延误'].fillna(0)
feature['出发机场天气得分'] = feature['出发机场天气得分'].interpolate()
feature['到达机场天气得分'] = feature['到达机场天气得分'].interpolate()
feature['平均延误时间'] = feature['平均延误时间'].interpolate()
feature['最大延误时间'] = feature['最大延误时间'].interpolate()
feature['延误时间中位数'] = feature['延误时间中位数'].interpolate()
feature['延误时间标准差'] = feature['延误时间标准差'].interpolate()

feature['本时段计划离港航班数'] = feature['本时段计划离港航班数'].fillna(feature['本时段计划离港航班数'].mean())
feature['本时段实际离港航班数'] = feature['本时段实际离港航班数'].fillna(feature['本时段实际离港航班数'].mean())
feature['本时段进港延误时长'] = feature['本时段进港延误时长'].fillna(feature['本时段进港延误时长'].mean())
feature['本时段实际进港延误航班数'] = feature['本时段实际进港延误航班数'].fillna(feature['本时段实际进港延误航班数'].mean())
feature['本时段离港延误时长'] = feature['本时段离港延误时长'].fillna(feature['本时段离港延误时长'].mean())
feature['本时段离港延误率'] = feature['本时段离港延误率'].fillna(feature['本时段离港延误率'].mean())
feature['上一时段进港延误时长'] = feature['上一时段进港延误时长'].fillna(feature['上一时段进港延误时长'].mean())
feature['上一时段离港延误时长'] = feature['上一时段离港延误时长'].fillna(feature['上一时段离港延误时长'].mean())
feature['上一时段实际进港航班数'] = feature['上一时段实际进港航班数'].fillna(feature['上一时段实际进港航班数'].mean())
feature['上一时段进港延误率'] = feature['上一时段进港延误率'].fillna(feature['上一时段进港延误率'].mean())

In [10]:
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()



In [17]:
#XGBOOST模型调参,测试样本0.3，训练集占0.7，模型效果为：
# XGBOOST模型评估
# 训练样本的准确率: 0.9724
# average_precision: 0.6494
# 查准率: 0.8938
# 召回率: 0.7025
# F1-Score: 0.7866
# AUC-Score_predprob: 0.9670
# 验证样本的准确率: 0.9707
# 查准率: 0.8790
# 召回率: 0.6891
# F1-Score: 0.7726
# AUC-Score: 0.9606
# ('ks', 0.6817467680926879)



#XGBOOST模型调参,测试样本0.1，训练集占0.9，模型效果为：
# XGBOOST模型评估
# 训练样本的准确率: 0.9718
# average_precision: 0.6414
# 查准率: 0.8901
# 召回率: 0.6960
# F1-Score: 0.7811
# AUC-Score_predprob: 0.9639
# 验证样本的准确率: 0.9710
# 查准率: 0.8856
# 召回率: 0.6887
# F1-Score: 0.7748
# AUC-Score: 0.9609
# ('ks', 0.6817358342785097)

from numpy import loadtxt,sort
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import pandas as pd
import numpy

dataset = feature.as_matrix()
X = dataset[:,1:36]
Y = dataset[:,36]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.9, random_state=7)
import numpy as np

# # # fit model on all training data
xgboost = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 tree_method='gpu_hist')


xgboost_model = xgboost.fit(x_train, y_train.astype('int'))


# 利用训练的模型来测试  训练集
prediction_train = xgboost_model.predict(x_train)
prediction_train_predprob = xgboost_model.predict_proba(x_train)[:,1]
 
# #利用训练的模型来测试  验证集集
prediction_test = xgboost_model.predict(x_test)
prediction_test_predprob = xgboost_model.predict_proba(x_test)[:,1]

prediction_test = prediction_test.tolist()
y_test = y_test.tolist()
prediction_test = np.array(prediction_test)
y_test = np.array([i for i in y_test])

y_train = y_train.tolist()
y_train = np.array([i for i in y_train])

prediction_train = prediction_train.tolist()
prediction_train = np.array(prediction_train)

# y_pred = gbdt_model.predict(x_test)
# y_pred = y_pred.tolist()
# y_test = y_test.tolist()
# y_pred = np.array(y_pred)
# y_test = np.array([i for i in y_test])
# accuracy = accuracy_score(y_test, y_pred)


print("XGBOOST模型评估")
testAccuracy = metrics.accuracy_score(y_train, prediction_train)
print("训练样本的准确率: %.4f" % testAccuracy)
 
average_precision = metrics.average_precision_score(y_train, prediction_train)
print("average_precision: %.4f" % average_precision)
 
#查准率
averagePrecisionScore = metrics.precision_score(y_train, prediction_train)
print("查准率: %.4f" % averagePrecisionScore)
 
##召回率
returnResultScore = metrics.recall_score(y_train, prediction_train)
print("召回率: %.4f" % returnResultScore)
 
##F1值
F1Score = metrics.f1_score(y_train, prediction_train)
print("F1-Score: %.4f" % F1Score)
 
#计算auc
roc_auc_predprob = metrics.roc_auc_score(y_train, prediction_train_predprob)
print("AUC-Score_predprob: %.4f" % roc_auc_predprob)
 

testAccuracy = metrics.accuracy_score(y_test, prediction_test)
print("验证样本的准确率: %.4f" % testAccuracy)
 
#查准率
averagePrecisionScore = metrics.precision_score(y_test, prediction_test)
print("查准率: %.4f" % averagePrecisionScore)
 
##召回率
returnResultScore = metrics.recall_score(y_test, prediction_test)
print("召回率: %.4f" % returnResultScore)
 
##F1值
F1Score = metrics.f1_score(y_test, prediction_test)
print("F1-Score: %.4f" % F1Score)
 
#计算auc
roc_auc = metrics.roc_auc_score(y_test, prediction_test_predprob)
print("AUC-Score: %.4f" % roc_auc)

ks_score = ks(prediction_test, y_test)
print(ks_score)




XGBOOST模型评估
训练样本的准确率: 0.9750
average_precision: 0.6820
查准率: 0.9154
召回率: 0.7230
F1-Score: 0.8079
AUC-Score_predprob: 0.9740
验证样本的准确率: 0.9698
查准率: 0.8694
召回率: 0.6850
F1-Score: 0.7663
AUC-Score: 0.9583
('ks', 0.6769884140022178)


In [12]:
import pickle
f= open('/Users/tuyu/000000毕业设计/Flight_Delay/Model/model/xgboost_model.pickle','wb')
pickle.dump(xgboost_model,f)
f.close()

In [11]:
import pickle
from numpy import loadtxt,sort
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import pandas as pd
import numpy

with open('/Users/tuyu/000000毕业设计/Flight_Delay/Model/model/xgboost_model.pickle','rb') as fr:
    xgboost_model = pickle.load(fr)

In [11]:
dataset = feature.as_matrix()
X = dataset[:,1:36]
Y = dataset[:,36]


/Users/tuyu/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [12]:
preds_offline = xgboost_model.predict(x_test)
preds_offline

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [13]:
prediction_test_predprob = xgboost_model.predict_proba(x_test)[:,1]
prediction_test_predprob

array([0.01961278, 0.0017142 , 0.00281581, ..., 0.00516581, 0.00811391,
       0.00312906], dtype=float32)

In [14]:
offline=(prediction_test_predprob >= 0.4) *1

In [16]:
# 利用训练的模型来测试  训练集
prediction_train = xgboost_model.predict(x_train)
prediction_train_predprob = xgboost_model.predict_proba(x_train)[:,1]
 
# #利用训练的模型来测试  验证集集
prediction_test = xgboost_model.predict(x_test)
prediction_test_predprob = xgboost_model.predict_proba(x_test)[:,1]

prediction_test = prediction_test.tolist()
y_test = y_test.tolist()
prediction_test = np.array(prediction_test)
y_test = np.array([i for i in y_test])

y_train = y_train.tolist()
y_train = np.array([i for i in y_train])

prediction_train = prediction_train.tolist()
prediction_train = np.array(prediction_train)

In [17]:
#召回率
returnResultScore = metrics.recall_score(offline, y_test)
returnResultScore

0.8477633309393451

In [18]:
from sklearn.metrics import classification_report 
print(classification_report(offline, y_test))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98   6445888
           1       0.73      0.85      0.79    428852

    accuracy                           0.97   6874740
   macro avg       0.86      0.91      0.88   6874740
weighted avg       0.97      0.97      0.97   6874740



In [ ]:
#XGBOOST原生接口
#划分数据集
# import time
# import xgboost as xgb
# start_time = time.time()
# offline = 0
# params={'booster':'gbtree',
#     'objective': 'binary:logistic',
#     'eval_metric':'auc',
#     'gamma':0.1,
#     'min_child_weight':1.1,
#     'max_depth':7,
#     'lambda':10,
#     'subsample':0.7,
#     'colsample_bytree':0.7,
#     'colsample_bylevel':0.7,
#     'eta': 0.01,
#     'tree_method':'exact',
#     'seed':1000,
#     'nthread':12,
#     'scale_pos_weight': 14
#     }

# params1={
# 'booster':'gbtree',
# 'objective': 'binary:logistic',
# 'scale_pos_weight': 3,

# 'gamma':0.2,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
# 'max_depth':8, # 构建树的深度，越大越容易过拟合
# 'lambda':3,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
# 'subsample':0.7, # 随机采样训练样本
# #'colsample_bytree':0.7, # 生成树时进行的列采样
# 'min_child_weight':3, 
# # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
# #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
# #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
# 'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
# 'eta': 0.03, # 如同学习率
# 'seed':1000,
# 'nthread':12,# cpu 线程数
# 'eval_metric': 'auc'
# }

# plst = list(params.items())
# num_rounds = 5000 # 迭代次数

# y = train.标签
# X = train.drop(['标签'],axis=1)

# val_y = val.标签
# val_X = val.drop(['标签'],axis=1)
# #val_X = val[feature_list]

# xgb_train = xgb.DMatrix(X, label=y)
# xgb_val = xgb.DMatrix(val_X,label=val_y)

# # return 训练和验证的错误率
# watchlist = [(xgb_train, 'train'),(xgb_val, 'val')]
# print("111")
# print ("跑到这里了xgb.train")
# # training model 
# # early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
# model = xgb.train(plst, xgb_train,num_boost_round=7000,evals=watchlist,early_stopping_rounds=500)
# print ("跑到这里了save_model")
# print ("best best_ntree_limit",model.best_ntree_limit)   #did not save the best,why?
# print ("best best_iteration",model.best_iteration) #get it?
# model.save_mdoel("D:/Desktop/xgb_model(原生接口).model")